Với code hiện tại khá dễ cho đóng gói. 
- Dưới backend ko thấy chỗ ra port hiện mặc định 5000
```app.py
from flask import Flask
from flask_cors import CORS
from api.scripts import bp as scripts_bp
from api.files import bp as files_bp

app = Flask(__name__)
CORS(app)

# Register blueprints
app.register_blueprint(scripts_bp, url_prefix='/api/scripts')
app.register_blueprint(files_bp, url_prefix='/api/files')

if __name__ == '__main__':
    app.run(debug=True)

```
- Ở frontend hiện đang nhận qua 
```api.js
import axios from 'axios';

const API_BASE_URL = 'http://localhost:5000';

// APi cho Upload file

export const uploadFile = async (file) => {
    if (!file) throw new Error('Please select a file first');
    
    const formData = new FormData();
    formData.append('file', file);

    const response = await axios.post(`${API_BASE_URL}/api/files/upload`, formData, {
        headers: {
            'Content-Type': 'multipart/form-data',
            'Accept': 'application/json'
        },
        responseType: 'json',
        timeout: 5000
    });
    
    return response;
};


// API cho run script
export const runScriptApi = async (scriptName) => {
    return axios.post(`${API_BASE_URL}/api/scripts/run/${scriptName}`);
}; 


//  API cho Output

// Lấy danh sách files từ output folder
export const getOutputFiles = async () => {
    const response = await fetch(`${API_BASE_URL}/api/files/list/output`);
    const data = await response.json();
    if (!data.success) throw new Error(data.error);
    return data.files;
};

// Download file từ output folder
export const downloadFile = (filename) => {
    window.open(`${API_BASE_URL}/api/files/download/output/${filename}`, '_blank');
};

// Download all files as zip from output folder
export const downloadAllAsZip = () => {
    window.open(`${API_BASE_URL}/api/files/download/output/all`, '_blank');
};

// API cho Example
export const getExampleFiles = async () => {
    const response = await fetch(`${API_BASE_URL}/api/files/list/example`);
    const data = await response.json();
    if (!data.success) throw new Error(data.error);
    return data.files;
};

// Download example file
export const downloadExampleFile = (filename) => {
    window.open(`${API_BASE_URL}/api/files/download/example/${filename}`, '_blank');
};
```

@backend @frontend   

Đóng gói Docker cho backend 25037:5000, Docker cho frontend 25038:3000, docker-compose cho cả 2. 

Với 2 option : 
http://localhost:5000 cho development và @http://103.253.20.13/ cho production 
- Biết @api.js  đang có const API_BASE_URL = 'http://localhost:5000';
- @api.js  nhận thông tin từ @app.py và các @routes.py @scripts.py 
-------------

NHỚ MỞ 0.0.0.0 để truy cập được với: @http://103.253.20.13/ 

I'll help you create Docker configurations for both frontend and backend services. Let's break this down into steps:

1. First, let's create the backend Dockerfile:

```dockerfile:backend/Dockerfile
FROM python:3.9-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

EXPOSE 5000

CMD ["python", "app.py"]
```

2. Create a frontend Dockerfile (modifying the existing one):

```dockerfile:frontend/Dockerfile
FROM node:18-alpine

WORKDIR /app

COPY package*.json ./
RUN npm install

COPY . .

# Create environment files for both development and production
RUN echo "REACT_APP_API_URL=http://localhost:5000" > .env.development && \
    echo "REACT_APP_API_URL=http://103.253.20.13:5000" > .env.production

RUN npm run build

EXPOSE 3000

CMD ["npm", "start"]
```

3. Create a docker-compose.yml file in the root directory:

```yaml:docker-compose.yml
version: '3.8'

services:
  backend:
    build: ./backend
    ports:
      - "25037:5000"
    environment:
      - FLASK_APP=app.py
      - FLASK_ENV=development
      - FLASK_RUN_HOST=0.0.0.0
    volumes:
      - ./backend:/app
    networks:
      - app-network

  frontend:
    build: ./frontend
    ports:
      - "25038:3000"
    environment:
      - NODE_ENV=development
    volumes:
      - ./frontend:/app
      - /app/node_modules
    depends_on:
      - backend
    networks:
      - app-network

networks:
  app-network:
    driver: bridge
```

4. Modify the backend/app.py to allow external access:

```python:backend/app.py
from flask import Flask
from flask_cors import CORS
from api.scripts import bp as scripts_bp
from api.files import bp as files_bp

app = Flask(__name__)
CORS(app)

# Register blueprints
app.register_blueprint(scripts_bp, url_prefix='/api/scripts')
app.register_blueprint(files_bp, url_prefix='/api/files')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)
```

5. Create a configuration file for the frontend to handle different environments:

```javascript:frontend/src/config.js
const config = {
  development: {
    API_BASE_URL: process.env.REACT_APP_API_URL || 'http://localhost:5000'
  },
  production: {
    API_BASE_URL: process.env.REACT_APP_API_URL || 'http://103.253.20.13:5000'
  }
};

export default config[process.env.NODE_ENV || 'development'];
```

6. Update the API service to use the config:

```javascript:frontend/src/services/api.js
import axios from 'axios';
import config from '../config';

const API_BASE_URL = config.API_BASE_URL;

// Rest of your existing api.js code remains the same
```

To deploy and run:

For development:
```bash
docker-compose up --build
```

For production:
```bash
docker-compose -f docker-compose.prod.yml up --build
```

Create a production docker-compose file:

```yaml:docker-compose.prod.yml
version: '3.8'

services:
  backend:
    build: ./backend
    ports:
      - "25037:5000"
    environment:
      - FLASK_APP=app.py
      - FLASK_ENV=production
      - FLASK_RUN_HOST=0.0.0.0
    networks:
      - app-network

  frontend:
    build: 
      context: ./frontend
      args:
        - NODE_ENV=production
    ports:
      - "25038:3000"
    environment:
      - NODE_ENV=production
    depends_on:
      - backend
    networks:
      - app-network

networks:
  app-network:
    driver: bridge
```

Don't forget to create a requirements.txt file in the backend directory:

```text:backend/requirements.txt
flask
flask-cors
pandas
python-dotenv
openai
requests
werkzeug
```

This setup will allow you to:
- Run the backend on port 25037 (mapped from container port 5000)
- Run the frontend on port 25038 (mapped from container port 3000)
- Switch between development and production environments
- Access the API from both localhost and the production IP
- Enable CORS and external access through 0.0.0.0